In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from pathlib import Path
from typing import List

import dataclass_array as dca
import matplotlib.pyplot as plt
import mitsuba as mi
import numpy as np
from PIL import Image
import pycolmap
import pyrender
import trimesh
import visu3d as v3d

import burybarrel.colmap_util as cutil

mi.set_variant("scalar_rgb")
# os.environ["PYOPENGL_PLATFORM"] = None

In [ ]:
# reconstr_path = Path("../results/barrelddt1-reconstr")
reconstr_path = Path("../results/barrel4-manual-reconstr")
# reconstr_path = Path("../results/dive1-depthcharge-16-09-trimend-reconstr")

# output_path = Path("../results/barrelddt1-reconstr/colmap-out/0")
output_path = reconstr_path / "colmap-out/0"
reconstruction = pycolmap.Reconstruction(output_path)
print(reconstruction.summary())

In [ ]:
p, c = cutil.get_pc(reconstruction)
pts3d = v3d.Point3d(p=p, rgb=c)

camlisttmp: List[v3d.Camera] = []
names = []
for img in reconstruction.images.values():
    spec = v3d.PinholeCamera.from_focal(resolution=(img.camera.height, img.camera.width), focal_in_px=img.camera.focal_length)
    T = v3d.Transform.from_matrix(img.cam_from_world.matrix()).inv
    camlisttmp.append(v3d.Camera(spec=spec, world_from_cam=T))
    names.append(img.name)
cams: v3d.Camera = dca.stack(camlisttmp)
v3d.make_fig([pts3d, cams])

In [20]:
W = cams.spec.resolution[1]
H = cams.spec.resolution[0]
renderer = pyrender.OffscreenRenderer(W, H)

for i, cam in enumerate(cams):
    K = cam.spec.K
    yfov = 2 * np.arctan((cam.spec.resolution[0] / 2) / cam.spec.K[0, 0])
    camera = pyrender.IntrinsicsCamera(K[0, 0], K[1, 1], K[0, 2], K[1, 2])
    # camera = pyrender.PerspectiveCamera(yfov=yfov, aspectRatio=W / H)
    T = mi.Transform4f(cam.world_from_cam.matrix4x4).rotate([1, 0, 0], 180).matrix.numpy()
    # T = mi.Transform4f().look_at(
    #         origin=[2.0, 2.0, 2.0],
    #         target=[0.0, 0.0, 0.0],
    #         up=[0.0, 0.0, 1.0],
    # ).matrix.numpy()
    # scenetrimesh = trimesh.load("../results/barrelddt1-reconstr/openmvs-out/scene_dense_mesh_refine.ply")
    scenetrimesh = trimesh.load(reconstr_path / "openmvs-out/scene_dense_mesh_refine.ply")
    # scenetrimesh = trimesh.load("../results/barrelddt1-reconstr/openmvs-out/scene_dense_mesh_refine_texture.ply")
    # scenetrimesh = trimesh.load("../../models3d/barrelsingle-scaled.ply")
    scenemesh = pyrender.Mesh.from_trimesh(scenetrimesh)
    # light = pyrender.PointLight(color=[1.0, 1.0, 1.0], intensity=500.0)
    light = pyrender.SpotLight(
        color=np.ones(3),
        intensity=500.0,
        innerConeAngle=np.pi / 16.0,
        outerConeAngle=np.pi / 6.0,
    )
    scene = pyrender.Scene(bg_color=[0, 0, 0], ambient_light=[0.02, 0.02, 0.02])
    scene.add(scenemesh, pose=np.eye(4))
    scene.add(camera, pose=T)
    scene.add(light, pose=T)
    color, depth = renderer.render(scene)
    img = Image.fromarray(color)
    name = Path(names[i]).stem
    (reconstr_path / "mesh-renders").mkdir(exist_ok=True)
    img.save(reconstr_path / f"mesh-renders/{name}_meshrender.png")
renderer.delete()

In [ ]:
fuze_trimesh = trimesh.load("../../models3d/barrelsingle-scaled.ply")
mesh = pyrender.Mesh.from_trimesh(fuze_trimesh)
camera = pyrender.PerspectiveCamera(yfov=np.pi / 3.0, aspectRatio=1.0)
T = mi.Transform4f().look_at(
        origin=[2.0, 0.0, 0.0],
        target=[0.0, 0.0, 0.0],
        up=[0.0, 0.0, 1.0],
).rotate([0,1,0], 180).matrix.numpy()
# T = np.array([
#     [1.0, 0, 0, 1],
#     [0, 1, 0, 0],
#     [0, 0, 1, 2],
#     [0, 0, 0, 1],
# ])
pl = pyrender.PointLight(color=[1.0, 1.0, 1.0], intensity=10.0)
scene = pyrender.Scene(bg_color=[0, 0, 0])
scene.add(mesh)
scene.add(camera, pose=T)
scene.add(pl, pose=T)
r = pyrender.OffscreenRenderer(400, 400)
color, depth = r.render(scene)
Image.fromarray(color)